In [ ]:
!pip install tensorflow-text

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 25.3 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/TFG/Pruebas ejecucion

In [ ]:
from model import Transformer, CustomSchedule, masked_loss, masked_acc
from data_preprocessing import create_datasets, text_vectorization, process_text

import tensorflow as tf
from tensorflow.keras import mixed_precision
import os

import nltk
import nltk.translate.bleu_score as bleu
nltk.download('punkt')

In [ ]:
MODEL_NAME = 'models/en_es4_256_512_8_0.2.h5'

In [ ]:
# Detect TPU and create a TPU strategy
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)
strategy = tf.distribute.TPUStrategy(resolver)

## Load model

In [ ]:
train, val, test = create_datasets()
context_text_processor, target_text_processor = text_vectorization(train)

# Wrap model creation and batch_translate function inside the strategy scope
with strategy.scope():
    #! Model
    num_layers = 6
    d_model = 256
    dff = 512
    num_heads = 8
    dropout_rate = 0.3

    learning_rate = CustomSchedule(d_model)
    my_optimizer = tf.keras.optimizers.legacy.Adam(learning_rate, beta_1=0.9, beta_2=0.98, epsilon=1e-9)

    model = Transformer(
        num_layers = num_layers,
        d_model = d_model,
        num_heads = num_heads,
        dff = dff,
        input_vocab_size = context_text_processor.vocabulary_size(),
        target_vocab_size = target_text_processor.vocabulary_size(),
        dropout_rate = dropout_rate,
    )

    model.compile(
        loss = masked_loss,
        optimizer=my_optimizer,
        metrics = [masked_acc],
    )

# Llama al modelo con datos ficticios para construirlo
dummy_input = tf.ones((1, 1), dtype=tf.int32)
dummy_context = tf.ones((1, 1), dtype=tf.int32)
model((dummy_context, dummy_input), training=False)

model.load_weights(MODEL_NAME)

## Inference

In [ ]:
def print_translation(sentence, translated_sentence, ground_truth):
  print(f'{"Input:":15s}: {sentence}')
  print(f'{"Prediction":15s}: {translated_sentence}')
  print(f'{"Ground truth":15s}: {ground_truth}')

In [ ]:
def detokenize_sentence(processor, sentence):
  output_sentence = tf.constant('')
  vocab = processor.get_vocabulary()
  lookup = tf.constant('')

  for token in sentence:
    output_sentence = tf.strings.join([output_sentence, vocab[token]], separator=' ')

  lookup = tf.strings.split(output_sentence, sep=' ')

  sentence = output_sentence.numpy().decode('utf-8')
  sentence = sentence.replace('[START]', '').replace('[END]', '').strip()

  return sentence, lookup

In [ ]:
def translate(sentence, context_processor, target_processor, model, max_tokens=128):
  # Convert the sentence to tensor
  sentence = tf.constant([[sentence]])
  assert isinstance(sentence, tf.Tensor)

  # Give the correct shape
  if len(sentence.shape) == 0:
    sentence = sentence[tf.newaxis]
  
  # Tokenize the sentence
  sentence = context_processor(sentence).to_tensor()

  # Input for the encoder
  encoder_input = sentence
  
  # Input for the decoder
  start_end_tokens = target_processor([''])[0]
  start_token = start_end_tokens[0][tf.newaxis]
  end_token = start_end_tokens[-1][tf.newaxis]

  # Convert decoder sentence to TensorArray for feed the model
  output_tensor = tf.TensorArray(dtype=tf.int64, size=0, dynamic_size=True)
  output_tensor = output_tensor.write(0, start_token)
  
  # seq_to_seq generation
  for i in tf.range(max_tokens):
    # Prepare the output tensor
    output = tf.transpose(output_tensor.stack())
    
    # Get the model predictions
    predictions = model((encoder_input, output), training=False)
    
    # Select the last token from the seq_len dimension
    predictions = predictions[:, -1, :] # Shape (batch_size, 1, vocab_size)
    # Get the most probable next token
    predicted_id = tf.argmax(predictions, axis=-1)

    # Concatenate the predicted token to the output sentence
    output_tensor = output_tensor.write(i+1, predicted_id) #TODO: en el tuto es predicted_id[0]

    # Check if the prediction is fully generated with the last token
    if predicted_id == end_token:
      break
  
  # Convert the output to the correct shape
  output = tf.transpose(output_tensor.stack()) # Shape (1, tokens_generated)

  # Detokenize the output sentence
  text, lookup = detokenize_sentence(target_processor, output[0])

  # 'tf.function' prevents us from using the attention_weights that were calculated on the
  # last iteration of the loop. So, recalculate them outside the loop.
  model((encoder_input, output[:, :-1]), training=False)
  attention_weigths = model.decoder.last_attn_scores
  #TODO: Check is attn_weigths is correct (with plotting function)

  return text, lookup, attention_weigths

In [ ]:
def batch_translate_single_batch(sentences, context_processor, target_processor, model, max_tokens=128):
    # Convert the sentences to a tensor
    sentences = tf.constant(sentences)
    assert isinstance(sentences, tf.Tensor)

    # Tokenize the sentences
    sentences = context_processor(sentences).to_tensor()

    # Input for the encoder
    encoder_input = sentences

    # Input for the decoder
    start_end_tokens = target_processor([''])[0]
    start_token = start_end_tokens[0][tf.newaxis]
    end_token = start_end_tokens[-1][tf.newaxis]
    end_token_id = end_token.numpy()[0]
    dot_token_id = target_processor.get_vocabulary().index('.')

    # Prepare the initial output tensor
    batch_size = tf.shape(encoder_input)[0]
    initial_output = tf.repeat(start_token, repeats=batch_size, axis=0)

    # Initialize the output tensor as a list of empty TensorArrays, one for each sentence
    output_tensors = [tf.TensorArray(dtype=tf.int64, size=0, dynamic_size=True) for _ in range(batch_size)]

    # Write the start token to each TensorArray
    for i, tensor_array in enumerate(output_tensors):
        tensor_array = tensor_array.write(0, initial_output[i])
        tensor_array.mark_used()
   

    # Initialize a boolean tensor to track the completed sentences
    completed_sentences = tf.zeros(batch_size, dtype=tf.bool)

    for _ in tf.range(max_tokens):
        # Prepare the output tensor
        max_sentence_length = max([tensor_array.size() for tensor_array in output_tensors])
        padded_output_tensors = [tf.pad(tensor_array.stack(), [[0, max_sentence_length - tensor_array.size()]]) for tensor_array in output_tensors]
        output = tf.stack(padded_output_tensors, axis=0)

        # Get the model predictions
        predictions = model((encoder_input, output), training=False)

        # Select the last token from the seq_len dimension
        predictions = predictions[:, -1, :]  # Shape (batch_size, 1, vocab_size)

        # Get the most probable next token
        predicted_ids = tf.argmax(predictions, axis=-1)

        # Concatenate the predicted token to the output sentence
        for i, (tensor_array, predicted_id) in enumerate(zip(output_tensors, predicted_ids)):
            if not completed_sentences[i]:
                if predicted_id.numpy() == end_token_id or predicted_id == dot_token_id:
                    completed_sentences = tf.where(tf.range(batch_size) == i, True, completed_sentences)
                    output_tensors[i] = tensor_array.write(tensor_array.size(), predicted_id)
                else:
                    output_tensors[i] = tensor_array.write(tensor_array.size(), predicted_id)
            else:
                # Keep the end token if the sentence is already completed
                output_tensors[i] = tensor_array.write(tensor_array.size(), end_token_id)

        # Check if all the predictions are fully generated with the last token
        if tf.reduce_all(completed_sentences):
            break
    

    # Convert the output to the correct shape
    output = tf.stack([tensor_array.stack() for tensor_array in output_tensors], axis=0)  # Shape (batch_size, tokens_generated)

    # Detokenize the output sentences
    texts, lookups = [], []
    for i in range(batch_size):
        text, lookup = detokenize_sentence(target_processor, output[i])
        texts.append(text)
        lookups.append(lookup)

    # 'tf.function' prevents us from using the attention_weights that were calculated on the
    # last iteration of the loop. So, recalculate them outside the loop.
    model((encoder_input, output[:, :-1]), training=False)
    attention_weights = model.decoder.last_attn_scores

    return texts, lookups, attention_weights

In [ ]:
def batch_translate(sentences, context_processor, target_processor, model, max_tokens=128, batch_size=16):
    num_batches = len(sentences) // batch_size + int(len(sentences) % batch_size > 0)
    all_translated_texts = []
    # all_lookups = []
    # all_attention_weights = []

    for i in range(num_batches):
        print('Batch starting... ' + str(i+1) + '/' + str(num_batches))
        batch_start = i * batch_size
        batch_end = min((i + 1) * batch_size, len(sentences))
        batch_sentences = sentences[batch_start:batch_end]
        
        # Call the previous batch_translate function with the smaller batch of sentences
        translated_text, lookup, attention_weights = batch_translate_single_batch(batch_sentences, context_processor, target_processor, model, max_tokens)
        
        all_translated_texts.extend(translated_text)
        # all_lookups.extend(lookup)
        # all_attention_weights.append(attention_weights)

    return all_translated_texts, _, __
    # return all_translated_texts, all_lookups, all_attention_weights

In [ ]:
# sentence = 'I want to be at home today'
# ground_truth = 'Quiero estar en casa hoy'/

# translated_text, lookup, attention_weigths = translate(sentence, context_text_processor, target_text_processor, model)
# print_translation(sentence, translated_text, ground_truth)

In [ ]:
# sentence = 'I like to drink water in the park'
# ground_truth = 'Me gusta beber agua en el parque'

# translated_text, lookup, attention_weigths = translate(sentence, context_text_processor, target_text_processor, model)
# print_translation(sentence, translated_text, ground_truth)

In [ ]:
# sentence = 'university has taught me that life is not only about technical knowledge, but also about meeting people who can help you in the future.'
# ground_truth = 'la universidad me ha enseñado que en la vida no solo importan los conocimientos técnicos, sino también conocer personas que te puedan ayudar en el futuro'

# translated_text, lookup, attention_weigths = translate(sentence, context_text_processor, target_text_processor, model)
# print_translation(sentence, translated_text, ground_truth)

In [ ]:
# print(context_text_processor.get_vocabulary()[:10])
# print(target_text_processor.get_vocabulary()[:10])

In [ ]:
# sentence = ['I want to be at home today', 'I like to drink water in the park', 'university has taught me that life is not only about technical knowledge, but also about meeting people who can help you in the future.']
# ground_truth = 'Quiero estar en casa hoy'

# translated_text, lookup, attention_weigths = batch_translate(sentence, context_text_processor, target_text_processor, model)
# print_translation(sentence, translated_text, ground_truth)

## Evaluation

In [ ]:
#open file E:\TFG\eval
with open('eval/europarl-v7.es-en.es', 'r') as f:
    data_es = f.readlines()

In [ ]:
with open('eval/europarl-v7.es-en.en', 'r') as f:
    data_en = f.readlines()

In [ ]:
def calculate_bleu_score(references, predictions):
    # Tokenize the sentences
    tokenized_references = [nltk.word_tokenize(ref) for ref in references]
    tokenized_predictions = [nltk.word_tokenize(pred) for pred in predictions]

    # Calculate the BLEU score using the NLTK library
    bleu_score = bleu.corpus_bleu([[ref] for ref in tokenized_references], tokenized_predictions)
    return bleu_score

In [ ]:
a = data_en[-128:]
b = data_es[-128:]

In [ ]:
translated_sentences, _, __ = batch_translate(a, context_text_processor, target_text_processor, model, batch_size=64)

In [ ]:
bleu_score = calculate_bleu_score(b, translated_sentences)
print("BLEU Score:", bleu_score)

In [ ]:
esp = [x.lower() for x in b]

In [ ]:
bleu_score = calculate_bleu_score(esp, translated_sentences)
print("BLEU Score:", bleu_score)